Adapted from
https://colab.research.google.com/github/huggingface/deep-rl-class/blob/master/notebooks/unit4/unit4.ipynb#scrollTo=V8oadoJSWp7C

In [1]:

%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip install pyvirtualdisplay
!pip install pyglet==1.5.1
!pip install imageio
!pip install gym[atari]
!pip install autorom[accept-rom-license]

In [2]:
# # Virtual display
# from pyvirtualdisplay import Display

# virtual_display = Display(visible=0, size=(1400, 900))
# virtual_display.start()

In [3]:
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit4/requirements-unit4.txt

  Cloning https://github.com/ntasfi/PyGame-Learning-Environment.git to /tmp/pip-req-build-d1ut4gvb
  Running command git clone --filter=blob:none --quiet https://github.com/ntasfi/PyGame-Learning-Environment.git /tmp/pip-req-build-d1ut4gvb
  Resolved https://github.com/ntasfi/PyGame-Learning-Environment.git to commit 3dbe79dc0c35559bb441b9359948aabf9bb3d331
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/simoninithomas/gym-games to /tmp/pip-req-build-52cyix__
  Running command git clone --filter=blob:none --quiet https://github.com/simoninithomas/gym-games /tmp/pip-req-build-52cyix__
  Resolved https://github.com/simoninithomas/gym-games to commit f31695e4ba028400628dc054ee8a436f28193f0b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing m

In [4]:
import numpy as np

from collections import deque

import matplotlib.pyplot as plt
%matplotlib inline

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

# Gym
import gym
import gym_pygame

# Hugging Face Hub
# from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.
import imageio

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
print(device)

cpu


In [7]:
env_id = "ALE/Pong-v5"
# Create the env
env = gym.make(env_id,obs_type="grayscale",full_action_space=False)

# Create the evaluation env
eval_env = gym.make(env_id,obs_type="grayscale",full_action_space=False)

# Get the state space and action space
s_size = gym.spaces.utils.flatten_space(env.observation_space).shape[0]
a_size = env.action_space.n

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]
/home/codespace/.python/current/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (210, 160)
  logger.warn(


In [8]:
print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample()) # Get a random observation

_____OBSERVATION SPACE_____ 

The State Space is:  33600
Sample observation [[246  42 173 ...  36 111  82]
 [186  40  39 ... 104 150 164]
 [230  13 156 ... 140 201   2]
 ...
 [114 229 101 ... 254 167 101]
 [ 51 134 139 ...  85  34 218]
 [113 253  91 ... 107 235 180]]


In [9]:
print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample()) # Take a random action


 _____ACTION SPACE_____ 

The Action Space is:  6
Action Space Sample 4


In [33]:
from os import listdir
from os.path import isfile, join

class Saver():
    """
    Class to save the output and input to an environment as tensors which can be used to train the nn
    """
    def __init__(self,save_dir,device="cpu") -> None:
        self.save_dir = save_dir

        self.observations = deque()#maxsize=3)
        self.actions = deque()#maxsize=1)
        self.rewards = deque()#maxsize=1)
        self.dones = deque()#maxsize=1)
        self.h=None
        self.w=None

        self.device = device

    def add_transition(self,observation, action, reward, done):
        """
        Observations: What the agent sees (dimensions are 210 x 160)
        Actions: What the agent does (dimensions are 6 x 1)
        Rewards: The reward the agent gets (range is -1 to 1)?
        Dones: If the episode is over (True or False)
        """
        self.observations.append(observation)
        self.actions.append(action)
        self.rewards.append(reward)
        self.dones.append(done)
        if len(self.observations) == 3:
            self._save_transitions()
            #clear screen of 3 screens ago.
            self.observations.popleft()
        
        #clear old values
        if len(self.actions) == 1:
            self.actions.popleft()
        if len(self.rewards) == 1:
            self.rewards.popleft()
        if len(self.dones) == 1:
            self.dones.popleft()


    def _convert_to_tensors(self):
        if self.h == None or self.w == None:
            h = self.observations[0].shape[0] #height of a pong image
            w = self.observations[0].shape[1] #width of a pong image

        #take 2 images as input, return one image (the model should need two input images to determine ball velocity)
        #expand action to a matrix the size of a pong image, and add it as a seperate channel in a tensor image later
        act = np.broadcast_to(self.actions[0], (h,w) )
        rew = self.rewards[0]
        don = self.dones[0]


        input = np.stack((self.observations[0], self.observations[1], act), axis=0) #put the channels together so my life is easier and the nn pays a lot of attention to act
        truth = (self.observations[2],rew,don) #tuple of outputs
        
        

        input = torch.tensor(input,dtype=torch.float)
        truth = (
            torch.tensor(np.expand_dims(truth[0],axis=0),device=self.device,dtype=torch.float),
            torch.unsqueeze(torch.tensor(truth[1],dtype=torch.float),dim=0),
            torch.unsqueeze(torch.tensor(truth[2],dtype=torch.float),dim=0),
        )
        assert input.shape == (3,h,w)
        assert truth[0].shape == (1,h,w)
        assert truth[1].shape == (1,), "shape is {}".format(truth[1].shape)
        assert truth[2].shape == (1,)

        return input,truth

    def _save_transitions(self):
        #change bool in dones to floats
        for i in range(len(self.dones)):
            self.dones[i] = float(self.dones[i])

        existing_files = [f for f in listdir(self.save_dir) if isfile(join(self.save_dir, f))]
        j = None
        if existing_files:
            #This grabs the largest integer out of all the filenames (filter the string for digit chars, convert those chars to an int)
            j = max([int(''.join([c for c in f if c.isdigit()])) for f in existing_files])
        else:
            j = -1
        j += 1

        input,truth = self._convert_to_tensors()
        torch.save(input,self.save_dir+"input{i}.pt".format(i=j))
        torch.save((truth),self.save_dir+"truth{i}.pt".format(i=j))

    

In [34]:
save_dir = "diffusion_training_data/"
saver = Saver(save_dir)

In [35]:
def run_pong(n_episodes, max_t):
    for i_episode in range(1, n_episodes+1):
        state = env.reset()[0]
        for t in range(max_t):
            action = env.action_space.sample()
            state, reward, terminated, truncated, info = env.step(action)
            saver.add_transition(state, action, reward, terminated or truncated)
            if terminated or truncated:
                break 
        

In [36]:
n_episodes = 10
max_t = 1000
run_pong(n_episodes,max_t)

KeyboardInterrupt: 